<a href="https://colab.research.google.com/github/ChloeMorgana/Dissertation-Project/blob/main/PubMedBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Loading of DrugProt dataset

Downloading and unzipping file

In [1]:
!wget https://zenodo.org/record/5042151/files/drugprot-gs-training-development.zip
!unzip drugprot-gs-training-development.zip

--2022-12-08 09:15:21--  https://zenodo.org/record/5042151/files/drugprot-gs-training-development.zip
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3796908 (3.6M) [application/octet-stream]
Saving to: ‘drugprot-gs-training-development.zip’

drugprot-gs-trainin 100%[===================>]   3.62M   411KB/s    in 9.2s    

2022-12-08 09:15:33 (402 KB/s) - ‘drugprot-gs-training-development.zip’ saved [3796908/3796908]

Archive:  drugprot-gs-training-development.zip
   creating: drugprot-gs-training-development/
  inflating: drugprot-gs-training-development/README.pdf  
   creating: drugprot-gs-training-development/development/
  inflating: drugprot-gs-training-development/development/drugprot_development_relations.tsv  
  inflating: drugprot-gs-training-development/development/drugprot_development_entities.tsv  
  inflating: drugprot-gs-training-development/d

Combining all three files together.

In [2]:
def loadDrugProt(abstracts_filename, entities_filename, relations_filename):

  docs = {}

  # Load the title/abstracts text in as documents
  with open(abstracts_filename, encoding='utf8') as f:
    for lineno,line in enumerate(f):
      split = line.strip('\n').split('\t')
      assert len(split) == 3, f"Expected 3 columns but got {len(split)} on line {lineno+1}"
      pubmed_id, title, abstract = split
      pubmed_id = int(pubmed_id)

      combined_text = title + '\n' + abstract
      docs[pubmed_id] = {'pubmed_id':pubmed_id, 'text':combined_text, 'entities':{}, 'relations':[]}

  # Load the entities and match them up with the documents
  with open(entities_filename, encoding='utf8') as f:
    for lineno,line in enumerate(f):
      split = line.strip('\n').split('\t')
      assert len(split) == 6, f"Expected 6 columns but got {len(split)} on line {lineno+1}"
      pubmed_id, entity_id, entity_type, start_coord, end_coord, entity_text = split

      pubmed_id = int(pubmed_id)
      start_coord = int(start_coord)
      end_coord = int(end_coord)

      assert pubmed_id in docs, f"Did not find matching document for pubmed_id={pubmed_id}"
      doc = docs[pubmed_id]

      assert doc['text'][start_coord:end_coord] == entity_text, f"Text for entity with coords {start_coord}:{end_coord} in document (pubmed_id={pubmed_id} does not match expected. 'f{doc['text'][start_coord:end_coord]}' != '{entity_text}'"

      entity = {'type':entity_type, 'start':start_coord, 'end':end_coord, 'text':entity_text}
      doc['entities'][entity_id] = entity

  if relations_filename is not None:
    # Load the relations and match them up with the entities in the corresponding document
    with open(relations_filename, encoding='utf8') as f:
      for lineno,line in enumerate(f):
        split = line.strip('\n').split('\t')
        assert len(split) == 4, f"Expected 4 columns but got {len(split)} on line {lineno+1}"

        pubmed_id, relation_type, arg1, arg2 = split

        pubmed_id = int(pubmed_id)
        assert arg1.startswith('Arg1:'), f"Relation argument should start with 'Arg1:'. Got: {arg1}"
        assert arg2.startswith('Arg2:'), f"Relation argument should start with 'Arg2:'. Got: {arg2}"

        # Remove arg1/arg2 from text
        arg1 = arg1.split(':')[1]
        arg2 = arg2.split(':')[1]

        assert pubmed_id in docs, f"Did not find matching document for pubmed_id={pubmed_id}"
        doc = docs[pubmed_id]

        assert arg1 in doc['entities'], f"Couldn't find entity with id={arg1} in document with pubmed_id={pubmed_id}"
        assert arg2 in doc['entities'], f"Couldn't find entity with id={arg2} in document with pubmed_id={pubmed_id}"

        relation = {'type':relation_type,'arg1':arg1,'arg2':arg2}
        doc['relations'].append(relation)

  # Convert the dictionary of documents (indexed by pubmed_id) into a simpler dictionary
  docs = sorted(docs.values(), key=lambda x:x['pubmed_id'])

  return docs

Applying function to dataset

In [3]:
docs = loadDrugProt('drugprot-gs-training-development/training/drugprot_training_abstracs.tsv',
                    'drugprot-gs-training-development/training/drugprot_training_entities.tsv',
                    'drugprot-gs-training-development/training/drugprot_training_relations.tsv')

Observation of doc structure


In [22]:
search = [ d for i,d in enumerate(docs) if d['pubmed_id'] == 1322791]
assert len(search) == 1, "Something went wrong and couldn't find the document we want"

doc = search[0]
doc

{'pubmed_id': 1322791,
 'text': 'Amsacrine and etoposide hypersensitivity of yeast cells overexpressing DNA topoisomerase II.\nIncreasing the cellular concentration of DNA topoisomerase II in yeast by expressing constitutively a plasmid-borne TOP2 gene encoding the enzyme greatly increases the sensitivity of the cells to amsacrine and etoposide (VP-16). This increased drug sensitivity at a higher intracellular DNA topoisomerase II level is observed in both RAD52+ repair-proficient strains and rad52 mutants that are defective in the repair of double-stranded breaks. These results provide strong support of the hypothesis that the cellular target of these drugs is DNA topoisomerase II, and that these drugs kill cells by converting DNA topoisomerase II into a DNA damaging agent.',
 'entities': {'T1': {'type': 'CHEMICAL',
   'start': 289,
   'end': 298,
   'text': 'amsacrine'},
  'T2': {'type': 'CHEMICAL', 'start': 303, 'end': 312, 'text': 'etoposide'},
  'T3': {'type': 'CHEMICAL', 'start':

We can observe the total number of relations across all documents below:

In [5]:
num = 0
for doc in docs:
  num += len(doc['relations'])

print(num)

17288


# Encoding Step

The code below imports certain packages that will be used during this relation extraction task.

In [6]:
#!pip install torch
!pip install transformers
!pip install segtok

#import transformers as ppb
from transformers import BertModel, AutoTokenizer
import pandas as pd
import numpy as np
import nltk
import torch
import itertools
from segtok.segmenter import split_single

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 8.5 MB/s 
     |████████████████████████████████| 182 kB 56.2 MB/s 
     |████████████████████████████████| 7.6 MB 42.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


This is a function that checks if a particular entity tag is part of a relation or not. This is used to form labels for the data so that the classifier can learn which sentences contain relations and which do not.

In [7]:
def isRelation(doc, key):

  relations = doc['relations']
  for r in relations:
    #Checks if the entity is involved in any part of a specified relation
    if (key == r['arg1']) or (key == r['arg2']):
      #Returns the name of the relation and a boolean value
      return True, r['type']
  return False, 0

def isEntityRelation(arg1,arg2,doc):
  for r in doc['relations']:
    if (arg1 == r['arg1']) and (arg2 == r['arg2']):
      return True
  return False


This function takes in the text and finds all of the labels for all pairs of entity relations. This will be used later to input into the classifier.

In [35]:
def find_sent_idxs(text):
  idxs = [0]
  idx = 0
  split_text = split_single(text)
  for t in split_text:
    idx += len(t)+1
    idxs.append(idx)
  return idxs

def find_entity_labels(idxs, keys, doc):
  labels = []
  entity_dict = {}

  for i in idxs:
    entity_dict[i]=[]

  for key in keys:
    for i in range(len(idxs)-1):
      if doc['entities'][key]['start'] >= idxs[i] and doc['entities'][key]['start'] <= idxs[i+1]:
        entity_dict[idxs[i]].append(key)

  for v in entity_dict.values():
    chems = []
    genes = []
    for name in v:
      if doc['entities'][name]['type'] == 'CHEMICAL':
        chems.append(name)
      else:
        genes.append(name)

    prods = itertools.product(chems,genes)
    for prod in prods:
      if isEntityRelation(prod[0],prod[1],doc):
        labels.append(1)
      else:
        labels.append(0)

  #print(labels)

  return labels



entity_idx = find_sent_idxs(docs[0]['text'])
print(entity_idx)
#print(find_entity_labels(entity_idx))


[0, 119, 240, 471, 590, 703, 861, 983, 1150, 1257, 1389, 1592, 1845]


This function adds in start and end entity tags to biomedical text. This ensures that every chemical mentioned is encased in [E1][/E1] tags, and every gene is encased in [E2][/E2] tags, so that we can extract the embeddings for these later.

In [55]:
def addEntityTags(docs):

  for doc in docs:
    #Contains the labels for each sentence (1 for positive result, 0 for no relation)
    doc['labels'] = []
    #Contains more explicit names for the relations.
    doc['classes'] = []

    str = ''
    index = 0

    #Contains every mention of chemicals and genes for a particular document
    keys = doc['entities'].keys()

    #Makes sure the indices of these mentions are in order
    key_list = sorted(keys, key= lambda x: (doc['entities'][x]['start']))

    #Finds the indexes of the start and end of each sentence in the text
    idxs = find_sent_idxs(doc['text'])


    l = find_entity_labels(idxs, key_list, doc)
    doc['entity_labels'] = l

    for k in key_list:
      #Adds a 1 to the list of labels if it is a positive result, 0 if negative
      b,r = isRelation(doc,k)
      if b:
        doc['labels'].append(1)
      else:
        doc['labels'].append(0)
      doc['classes'].append(r)
      v = doc['entities'][k]

      #Start and end indexes of entity
      start = v['start']
      end = v['end']

      if v['type'] == 'CHEMICAL':
        str += doc['text'][index:start] + '[E1] ' + doc['text'][start:end] + ' [/E1]'
      else:
        str += doc['text'][index:start] + '[E2] ' + doc['text'][start:end] + ' [/E2]'
      index = end
    
    #Adds the tagged text to the dictionary of a particular document
    doc['tagged_text'] = str 

addEntityTags(docs)
#print(docs[1]['tagged_text'])
#print(docs[0]['entity_labels'])

print(docs[0].keys())
print(docs[0]['entities'])
print(docs[0]['entity_labels'])
print(docs[0]['tagged_text'])



dict_keys(['pubmed_id', 'text', 'entities', 'relations', 'labels', 'classes', 'tagged_text', 'entity_labels'])
{'T1': {'type': 'CHEMICAL', 'start': 1455, 'end': 1466, 'text': 'benzamidine'}, 'T2': {'type': 'CHEMICAL', 'start': 1489, 'end': 1493, 'text': 'EACA'}, 'T3': {'type': 'CHEMICAL', 'start': 292, 'end': 298, 'text': 'lysine'}, 'T4': {'type': 'CHEMICAL', 'start': 308, 'end': 334, 'text': 'epsilon-amino caproic acid'}, 'T5': {'type': 'CHEMICAL', 'start': 336, 'end': 340, 'text': 'EACA'}, 'T6': {'type': 'CHEMICAL', 'start': 400, 'end': 406, 'text': 'lysine'}, 'T7': {'type': 'GENE-Y', 'start': 1165, 'end': 1170, 'text': 'gp330'}, 'T8': {'type': 'GENE-Y', 'start': 1172, 'end': 1179, 'text': 'plasmin'}, 'T9': {'type': 'GENE-Y', 'start': 227, 'end': 238, 'text': 'plasminogen'}, 'T10': {'type': 'GENE-Y', 'start': 1236, 'end': 1255, 'text': 'alpha 2-antiplasmin'}, 'T11': {'type': 'GENE-Y', 'start': 1272, 'end': 1279, 'text': 'plasmin'}, 'T12': {'type': 'GENE-Y', 'start': 1283, 'end': 1288

Next, we can reduce the text to only sentences potentially containing relations between chemicals and genes by splitting the sentences and seeing if there are any entity tags present within the sentence. The findLabels function goes through the list of labels for a particular document and sees if there is a relation present within that sentence.

(Note for later: I didn't take into account relations across sentences in this implementation, will have to change something to ensure that both entities for one relation are in the same sentence)

In [40]:
from collections import Counter

def relevantText():

  target_text = []
  labels = []

  for doc in docs:
    #print(doc.keys())
    txt = doc['tagged_text']
    lbls = doc['labels']

    split_text = []
    split_text += split_single(txt)
    offset = 0

    for s in split_text:
      if '[/E' in s:
        o, l, new_lbls = findLabels(s,lbls,offset)
        target_text.append(s)
        lbls = new_lbls
        offset += o
        labels.append(l)

  return target_text, labels

def findLabels(text, labels, offset=0):
    words = text.split(" ")
    counter = Counter(words)
    idx = counter['[E2]']+counter['[E1]']
    isRel = False
    for i in range(idx):
      if labels[i]==1:
        isRel = True
    offset+=idx
    if isRel:
      label = 1
    else:
      label = 0
    l = labels[idx:]
    return offset, label, l

target_text, labels= relevantText()

#Both the target text and the labels must be the same length
print(len(target_text))
print(len(labels))


29234
29234


There are around 17288 total relations, therefore there some sentences that contain entities but don't actually contain a relation. The code below shows a snippet of the target text which should only contain sentences that contain tagged entities. It also contains the first five labels which are all 0 in this case. These seems sensible as there isn't a chemical in the first sentence, and therefore there is no relationship specified between a chemical and gene.

In [41]:
print(target_text[:5])
print(labels[:5])

['Analysis of [E2] plasmin [/E2] binding and [E2] urokinase [/E2] activation of [E2] plasminogen [/E2] bound to the [E2] Heymann nephritis autoantigen [/E2], [E2] gp330 [/E2].', 'Previously, we demonstrated that the [E2] Heymann nephritis autoantigen [/E2], [E2] gp330 [/E2], can serve as a receptor site for [E2] plasminogen [/E2].', 'This binding was not significantly inhibited by the [E1] lysine [/E1] analogue [E1] epsilon-amino caproic acid [/E1] ([E1] EACA [/E1]), indicating that [E2] plasminogen [/E2] binding was not just through [E1] lysine [/E1] binding sites as suggested for other [E2] plasminogen [/E2] binding sites.', 'We now report that once [E2] plasminogen [/E2] is bound to [E2] gp330 [/E2], it can be converted to its active form of [E2] plasmin [/E2] by [E2] urokinase [/E2].', 'This conversion of [E2] plasminogen [/E2] to [E2] plasmin [/E2] proceeds at a faster rate when [E2] plasminogen [/E2] is first prebound to [E2] gp330 [/E2].']
[0, 0, 0, 0, 0]


# Embedding Step

Next, a BERT model is defined that has been pre-trained using PubMed texts. This allows BERT to recognise biomedical terms within the text and understand their context better.

The tokeniser is also defined using pubmed texts, and special tokens are added in so that BERT can recognise them and not break them down when tokenising.

In [42]:
model = BertModel.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext",
                                  output_hidden_states = True,
                                  )

tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext") 

tokenizer.add_tokens(["[E1]", "[/E1]", "[E2]", "[/E2]"])

tokenizer.vocab["[E1]"]

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

30522

This function adds [CLS] and [SEP] tokens to the beginning and end of sentences so that BERT knows when a new sentence is beginning. It then tokenises these sentences and converts all of the inputs into tensors.

In [92]:
def bert_text_prep(text, tokenizer):

  #CLS lets BERT know when start of sentence begins, SEP indicates start of second sentence.
  marked_text = "[CLS]" + text + "[SEP]"

  #Tokenizes text
  tokenized_text = tokenizer.tokenize(marked_text)

  if len(tokenized_text)>512:
    print(text)

  #Converts to ids
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

  segments_ids = [1]*len(indexed_tokens)

  # Converts inputs to tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  return tokenized_text, tokens_tensor, segments_tensors

These tensors are then used to find the embeddings of all of the tokens.

In [93]:
def get_bert_embeddings(tokens_tensor, segments_tensors, model):
  with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2][1:]

  token_embeddings = hidden_states[-1]
  #Collapsing tensor into 1 dimension
  token_embeddings = torch.squeeze(token_embeddings, dim=0)
  #converting tensors to lists
  list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]

  return list_token_embeddings

In order to find the relevant embeddings (entity start tag embeddings), findTags looks through the tokenised sentence and adds the index of that particular tag into a list of chemical indices or gene indices.

These are then used after the sentence has been converted into embeddings to find the embedding at these specific indexes. The embeddings are accompanied by the cartesian product of all of the chemical and gene entities found, so that the embeddings for the specified indexes are concatenated.

In [94]:
def findTags(tokens):
  chems = []
  genes = []

  for i in range(len(tokens)):
    #Adds the indexes to the relevant lists
    if tokens[i] == '[e1]':
      chems.append(i)
    elif tokens[i] == '[e2]':
      genes.append(i)
  
  #Returns empty lists if either chemicals or genes aren't present
  if (chems == []) or (genes == []):
    return [], []
  else:
    return chems, genes

def findEmbeddings(prods, embeddings):
  #Concatenates the embeddings
  entity_embeddings = []
  for elt in prods:
    if elt != ():
      chem = embeddings[elt[0]]
      gene = embeddings[elt[1]]
      entity_embeddings.append(chem+gene)
  return entity_embeddings


Getting CLS and entity embeddings for each sentence



In [95]:
#t = target_text[:11]
#l = labels[:11]
cls_embeddings = []
entity_embeddings = []

for text in target_text:

    #Finds tokenized text and tensors for a particular sentence in a document
    tokenized_text, tokens_tensor, segments_tensors = bert_text_prep(text, tokenizer)

    #Finds the embeddings
    #list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)

    #Adds the [CLS] token embedding to the list
    #cls_embeddings.append(list_token_embeddings[0])

    #Finds the indexes for chemicals and genes in the sentence
    #chems, genes = findTags(tokenized_text)

    #Finds the cartesian product of all of these chemicals and genes
    #prods = itertools.product(chems,genes)

    #Adds the concatenated embeddings of all chemicals and genes to the list
    #entity_embeddings += findEmbeddings(prods, list_token_embeddings)

np.save("cls_embeddings.npy",cls_embeddings)
np.save("entity_embeddings.npy",entity_embeddings)
#print(len(entity_embeddings))
#print(len(cls_embeddings))

In the present studies, the effects of [E2] CB(1) [/E2] agonists [[E1] (6aR,10aR)-3-(1-adamantyl)-6,6,9-trimethyl-6a,7,10,10a-tetrahydrobenzo[c]chromen-1-ol [/E1] ([E1] AM411 [/E1]), [E1] 9β-(hydroxymethyl)-3-(1-adamantyl)-hexahydrocannabinol [/E1] ([E1] AM4054 [/E1]), [E1] R-(+)-[2,3-dihydro-5-methyl-3-[(morpholinyl)methyl]pyrrolo[1,2,3-de]-1,4-benzoxazinyl]-(1-naphthalenyl)methanone mesylate [/E1] ([E1] WIN55,212.2 [/E1]), [E1] Δ(9)-tetrahydrocannabinol [/E1] ([E1] Δ(9)-THC [/E1]), [E1] (R)-(+)-arachidonyl-1'-hydroxy-2'-propylamide (methanandamide) [/E1]], [E2] CB(1) [/E2] antagonists [[E1] 5-(4-chlorophenyl)-1-(2,4-dichloro-phenyl)-4-methyl-N-(piperidin-1-yl)-1H-pyrazole-3-carboxamide [/E1] ([E1] SR141716A [/E1]), [E1] 5-(4-alkylphenyl)-1-(2,4-dichlorophenyl)-4-methyl-N-(piperidin-1-yl)-1H-pyrazole-3-carboxamide [/E1] ([E1] AM4113 [/E1])], and [E1] dopamine [/E1] (DA)-related [[E1] methamphetamine [/E1], [E1] (±)-6-chloro-7,8-dihydroxy-3-allyl-1-phenyl-2,3,4,5-tetrahydro-1H-3-benzaz

## Classification Step

Relevant imports:



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix

Splitting the embeddings into train/test sets

In [ ]:
#Splits the sentences into training and test sets
cls_train_features, cls_test_features, cls_train_labels, cls_test_labels = train_test_split(cls_embeddings, labels)

Classifying these using logistic regression

In [ ]:
#Defines a logistic regression classifier
lr_clf = LogisticRegression()

#Fits the model to the training data
lr_clf.fit(cls_train_features, cls_train_labels)

#Tests the model using the test data
lr_clf.score(cls_test_features, cls_test_labels)

Entity start architecture

In [ ]:
ent_labels = [doc['entity_labels'] for doc in docs if 'entity_labels' in doc.keys()]
e = list(itertools.chain.from_iterable(ent_labels))
print(e)
print(len(e))

start_train_features, start_test_features, start_train_labels, start_test_labels = train_test_split(entity_embeddings, e)

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(start_train_features, start_train_labels)
lr_clf.score(start_test_features, start_test_labels)

Establishing the relation labels

In [ ]:
labels = list(set([r['type'] for doc in docs for r in doc['relations']]))